In [74]:
from dotenv import load_dotenv
import os
load_dotenv()

# Test API key
API_KEY = os.getenv("TMDB_API_KEY")
print("API Key loaded successfully!" if API_KEY else " API Key NOT found!")

# Test imports
import pandas as pd
import requests
print("All packages imported successfully!")

API Key loaded successfully!
All packages imported successfully!


Fetch Data

In [75]:
movie_ids = [
    0, 299534, 19995, 140607, 299536, 597, 135397,
    420818, 24428, 168259, 99861, 284054, 12445,
    181808, 330457, 351286, 109445, 321612, 260513
]

import requests
import time

def fetch_movie(movie_id):
    url = f"https://api.themoviedb.org/3/movie/{movie_id}"
    params = {
        'api_key': API_KEY,
        'language': 'en-US'
    }
    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Failed to fetch movie_id {movie_id}: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error fetching movie_id {movie_id}: {e}")
        return None

# --- STEP 3: Fetch all movies ---
print("Fetching movie data from TMDb...\n")
raw_movies = []

for mid in movie_ids:
    if mid <= 0:  # Skip invalid IDs like 0
        print(f"Skipping invalid movie_id: {mid}")
        continue
    movie = fetch_movie(mid)
    if movie:
        raw_movies.append(movie)
    time.sleep(0.25)  # Respect API rate limits

# --- STEP 4: Create DataFrame ---
df = pd.DataFrame(raw_movies)

# --- STEP 5: Save raw data (optional) ---
df.to_csv("tmdb_raw_data.csv", index=False)

print(f"\nSuccessfully fetched {len(df)} movies.")
print("\nChecking nested fields:")
print(df[['title', 'genres', 'production_companies', 'belongs_to_collection']].head())


Fetching movie data from TMDb...

Skipping invalid movie_id: 0

Successfully fetched 18 movies.

Checking nested fields:
                          title  \
0             Avengers: Endgame   
1                        Avatar   
2  Star Wars: The Force Awakens   
3        Avengers: Infinity War   
4                       Titanic   

                                              genres  \
0  [{'id': 12, 'name': 'Adventure'}, {'id': 878, ...   
1  [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...   
2  [{'id': 12, 'name': 'Adventure'}, {'id': 28, '...   
3  [{'id': 12, 'name': 'Adventure'}, {'id': 28, '...   
4  [{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...   

                                production_companies  \
0  [{'id': 420, 'logo_path': '/hUzeosd33nzE5MCNsZ...   
1  [{'id': 444, 'logo_path': None, 'name': 'Dune ...   
2  [{'id': 1, 'logo_path': '/tlVSws0RvvtPBwViUyOF...   
3  [{'id': 420, 'logo_path': '/hUzeosd33nzE5MCNsZ...   
4  [{'id': 4, 'logo_path': '/jay6WcMgagAklUt7i9Eu..

View Data

In [76]:
# Let's check what columns we actually have and what data exists
print("Checking what data we actually received...\n")

print("All columns in DataFrame:")
print(df.columns.tolist())

print("DataFrame Info:")
print(df.info())

print("Sample of first movie (all columns):")
print(df.iloc[0].to_dict())

print("Check which columns have non-null data:")
print(df.notna().sum().sort_values(ascending=False))

Checking what data we actually received...

All columns in DataFrame:
['adult', 'backdrop_path', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id', 'imdb_id', 'origin_country', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'video', 'vote_average', 'vote_count']
DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  18 non-null     bool   
 1   backdrop_path          18 non-null     object 
 2   belongs_to_collection  16 non-null     object 
 3   budget                 18 non-null     int64  
 4   genres                 18 non-null     object 
 5   homepage               18 non-null     object 
 6   id              

In [77]:
# Let's explore the DataFrame structure
print("DataFrame shape:", df.shape)
print("\nColumn names:")
print(df.columns.tolist())
print("\nData types:")
print(df.dtypes)
print("\nFirst row sample:")
display(df.iloc[0])

DataFrame shape: (18, 26)

Column names:
['adult', 'backdrop_path', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id', 'imdb_id', 'origin_country', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'video', 'vote_average', 'vote_count']

Data types:
adult                       bool
backdrop_path             object
belongs_to_collection     object
budget                     int64
genres                    object
homepage                  object
id                         int64
imdb_id                   object
origin_country            object
original_language         object
original_title            object
overview                  object
popularity               float64
poster_path               object
production_companies      object
production_countries      object
release_date              object
revenue    

adult                                                                False
backdrop_path                             /7RyHsO4yDXtBv1zUU3mTpHeQ0d5.jpg
belongs_to_collection    {'id': 86311, 'name': 'The Avengers Collection...
budget                                                           356000000
genres                   [{'id': 12, 'name': 'Adventure'}, {'id': 878, ...
homepage                    https://www.marvel.com/movies/avengers-endgame
id                                                                  299534
imdb_id                                                          tt4154796
origin_country                                                        [US]
original_language                                                       en
original_title                                           Avengers: Endgame
overview                 After the devastating events of Avengers: Infi...
popularity                                                         16.8333
poster_path              

Data Cleaning (Dropping Columns)

In [78]:
columns_to_drop = ['adult', 'imdb_id', 'original_title', 'video', 'homepage']

# Check which columns actually exist in our DataFrame
existing_columns_to_drop = [col for col in columns_to_drop if col in df.columns]
missing_columns = [col for col in columns_to_drop if col not in df.columns]

print(f"\nColumns that will be dropped: {existing_columns_to_drop}")
if missing_columns:
    print(f"Columns not found in DataFrame: {missing_columns}")

# Drop the columns
df = df.drop(columns=existing_columns_to_drop)

print(f"\nNumber of columns: {len(df.columns)}")
print("Remaining columns:", df.columns.tolist())


Columns that will be dropped: ['adult', 'imdb_id', 'original_title', 'video', 'homepage']

Number of columns: 21
Remaining columns: ['backdrop_path', 'belongs_to_collection', 'budget', 'genres', 'id', 'origin_country', 'original_language', 'overview', 'popularity', 'poster_path', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'vote_average', 'vote_count']


Data Cleaning(Parsing the nested JSON columns)

In [79]:
# Let's investigate the actual structure of the data
print("Investigating data structure...\n")

# Check a few rows to see what the data actually looks like
print("Sample of 'belongs_to_collection':")
print(df['belongs_to_collection'].head())
print(f"\nType: {type(df['belongs_to_collection'].iloc[0])}")
print(f"Value: {df['belongs_to_collection'].iloc[0]}")

print("\n" + "="*60)
print("Sample of 'genres':")
print(df['genres'].head())
print(f"\nType: {type(df['genres'].iloc[0])}")
print(f"Value: {df['genres'].iloc[0]}")

print("\n" + "="*60)
print("Sample of 'production_companies':")
print(df['production_companies'].head())
print(f"\nType: {type(df['production_companies'].iloc[0])}")

# Let's also check if any of these columns have non-null values
print("\n" + "="*60)
print("Non-null counts:")
print(df[['belongs_to_collection', 'genres', 'spoken_languages', 
         'production_countries', 'production_companies']].notna().sum())

Investigating data structure...

Sample of 'belongs_to_collection':
0    {'id': 86311, 'name': 'The Avengers Collection...
1    {'id': 87096, 'name': 'Avatar Collection', 'po...
2    {'id': 10, 'name': 'Star Wars Collection', 'po...
3    {'id': 86311, 'name': 'The Avengers Collection...
4                                                 None
Name: belongs_to_collection, dtype: object

Type: <class 'dict'>
Value: {'id': 86311, 'name': 'The Avengers Collection', 'poster_path': '/yFSIUVTCvgYrpalUktulvk3Gi5Y.jpg', 'backdrop_path': '/zuW6fOiusv4X9nnW3paHGfXcSll.jpg'}

Sample of 'genres':
0    [{'id': 12, 'name': 'Adventure'}, {'id': 878, ...
1    [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...
2    [{'id': 12, 'name': 'Adventure'}, {'id': 28, '...
3    [{'id': 12, 'name': 'Adventure'}, {'id': 28, '...
4    [{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...
Name: genres, dtype: object

Type: <class 'list'>
Value: [{'id': 12, 'name': 'Adventure'}, {'id': 878, 'name': 'Science Fiction'}, {'id'

In [80]:
import numpy as np
import json
import ast

print("Parsing nested JSON columns...\n")

#  Extract Collection Name 
print("1. Extracting collection name from 'belongs_to_collection'...")
def extract_collection_name(collection):
    """Extract collection name from belongs_to_collection"""
    if collection is None or (isinstance(collection, float) and np.isnan(collection)):
        return None
    if isinstance(collection, dict):
        return collection.get('name', None)
    return None

df['belongs_to_collection'] = df['belongs_to_collection'].apply(extract_collection_name)
print(f"   Collection names extracted. Non-null values: {df['belongs_to_collection'].notna().sum()}")

# Extract Genre Names 
print("\n2. Extracting genre names from 'genres'...")
def extract_names(json_list, key='name'):
    """Extract names from a list of dictionaries and join with |"""
    # Handle None and NaN
    if json_list is None:
        return None
    if isinstance(json_list, float) and np.isnan(json_list):
        return None
    if json_list == [] or json_list == "":
        return None
    
    # If it's already a list, process it
    if isinstance(json_list, list):
        if len(json_list) == 0:
            return None
        names = [item.get(key, '') for item in json_list if isinstance(item, dict)]
        return '|'.join(names) if names else None
    
    # If it's a string, try to parse it
    if isinstance(json_list, str):
        try:
            json_list = json.loads(json_list)
        except json.JSONDecodeError:
            try:
                json_list = ast.literal_eval(json_list)
            except (ValueError, SyntaxError):
                return None
        
        # Now process the parsed list
        if isinstance(json_list, list) and len(json_list) > 0:
            names = [item.get(key, '') for item in json_list if isinstance(item, dict)]
            return '|'.join(names) if names else None
    
    return None

df['genres'] = df['genres'].apply(extract_names)
print(f"   Genres extracted. Sample: {df['genres'].dropna().head(3).tolist()}")

# --- 2.2.3: Extract Spoken Languages ---
print("\n3. Extracting spoken languages...")
df['spoken_languages'] = df['spoken_languages'].apply(lambda x: extract_names(x, key='english_name'))
print(f"   Languages extracted. Sample: {df['spoken_languages'].dropna().head(3).tolist()}")

# --- 2.2.4: Extract Production Countries ---
print("\n4. Extracting production countries...")
df['production_countries'] = df['production_countries'].apply(extract_names)
print(f"   Countries extracted. Sample: {df['production_countries'].dropna().head(3).tolist()}")

# --- 2.2.5: Extract Production Companies ---
print("\n5. Extracting production companies...")
df['production_companies'] = df['production_companies'].apply(extract_names)
print(f"   Companies extracted. Sample: {df['production_companies'].dropna().head(3).tolist()}")

# --- Inspect extracted columns ---

print("INSPECTION: Value counts for extracted columns")

print("\nGenres distribution:")
print(df['genres'].value_counts().head(10))

print("\nCollection distribution:")
print(df['belongs_to_collection'].value_counts().head(10))

print("\nJSON parsing complete!")

Parsing nested JSON columns...

1. Extracting collection name from 'belongs_to_collection'...
   Collection names extracted. Non-null values: 16

2. Extracting genre names from 'genres'...
   Genres extracted. Sample: ['Adventure|Science Fiction|Action', 'Action|Adventure|Fantasy|Science Fiction', 'Adventure|Action|Science Fiction']

3. Extracting spoken languages...
   Languages extracted. Sample: ['English|Japanese|Xhosa', 'English|Spanish', 'English']

4. Extracting production countries...
   Countries extracted. Sample: ['United States of America', 'United States of America|United Kingdom', 'United States of America']

5. Extracting production companies...
   Companies extracted. Sample: ['Marvel Studios', 'Dune Entertainment|Lightstorm Entertainment|20th Century Fox|Ingenious Media', 'Lucasfilm Ltd.|Bad Robot']
INSPECTION: Value counts for extracted columns

Genres distribution:
genres
Adventure|Action|Science Fiction             3
Action|Adventure|Science Fiction|Thriller    2
Ac

In [81]:
df[['belongs_to_collection', 'genres', 'spoken_languages', 'production_countries', 'production_companies']].head(3)


,belongs_to_collection,genres,spoken_languages,production_countries,production_companies
0,The Avengers Collection,Adventure|Science Fiction|Action,English|Japanese|Xhosa,United States of America,Marvel Studios
1,Avatar Collection,Action|Adventure|Fantasy|Science Fiction,English|Spanish,United States of America|United Kingdom,Dune Entertainment|Lightstorm Entertainment|20...
2,Star Wars Collection,Adventure|Action|Science Fiction,English,United States of America,Lucasfilm Ltd.|Bad Robot


Data Cleaning (Handling Datatypes ,Duplicates and Filtering )

In [82]:
print("   Converting 'budget', 'id', 'popularity' to numeric...")
df['budget'] = pd.to_numeric(df['budget'], errors='coerce')
df['id'] = pd.to_numeric(df['id'], errors='coerce')
df['popularity'] = pd.to_numeric(df['popularity'], errors='coerce')
df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')
df['runtime'] = pd.to_numeric(df['runtime'], errors='coerce')
df['vote_count'] = pd.to_numeric(df['vote_count'], errors='coerce')
df['vote_average'] = pd.to_numeric(df['vote_average'], errors='coerce')

# Convert release_date to datetime
print("   Converting 'release_date' to datetime...")
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

print("Data types converted successfully!")
print("\n Replacing unrealistic/placeholder values...")

# Replace 0 values with NaN for budget, revenue, runtime
print("Replacing 0 values with NaN for budget, revenue, runtime...")
df.loc[df['budget'] == 0, 'budget'] = np.nan
df.loc[df['revenue'] == 0, 'revenue'] = np.nan
df.loc[df['runtime'] == 0, 'runtime'] = np.nan

print(f"   - Budget: {df['budget'].isna().sum()} null values")
print(f"   - Revenue: {df['revenue'].isna().sum()} null values")
print(f"   - Runtime: {df['runtime'].isna().sum()} null values")

# Convert Budget and Revenue to Millions ---
print("\n Converting budget and revenue to millions of USD...")
df['budget_musd'] = df['budget'] / 1_000_000
df['revenue_musd'] = df['revenue'] / 1_000_000

print(f"   Created 'budget_musd' and 'revenue_musd' columns")
print(f"   Sample budget (M USD): {df['budget_musd'].dropna().head(3).tolist()}")
print(f"   Sample revenue (M USD): {df['revenue_musd'].dropna().head(3).tolist()}")

# --- 2.3.4: Replace Placeholder Text with NaN ---
print("\n Replacing placeholder text in 'overview' and 'tagline'...")

# Common placeholders
placeholders = ['No Data', 'no data', 'N/A', 'n/a', 'Unknown', 'TBD', '']

def replace_placeholders(text):
    """Replace placeholder text with NaN"""
    if pd.isna(text) or text is None:
        return np.nan
    if isinstance(text, str):
        text_stripped = text.strip()
        if text_stripped in placeholders or text_stripped == '':
            return np.nan
    return text

df['overview'] = df['overview'].apply(replace_placeholders)
df['tagline'] = df['tagline'].apply(replace_placeholders)

print(f"   - Overview null values: {df['overview'].isna().sum()}")
print(f"   - Tagline null values: {df['tagline'].isna().sum()}")

# Remove Duplicates
print("\n Removing duplicates...")
initial_rows = len(df)
df = df.drop_duplicates(subset=['id'])
duplicates_removed = initial_rows - len(df)
print(f"Removed {duplicates_removed} duplicate rows based on id")

# Drop Rows with Missing ID or Title 
print("\n Dropping rows with missing 'id' or 'title'...")
initial_rows = len(df)
df = df.dropna(subset=['id', 'title'])
rows_removed = initial_rows - len(df)
print(f"Removed {rows_removed} rows with missing id/title")

# Keep Only Rows with ≥10 Non-Null Columns 
print("\n Keeping only rows with at least 10 non-null columns...")
initial_rows = len(df)
df = df[df.notna().sum(axis=1) >= 10]
rows_removed = initial_rows - len(df)
print(f"Removed {rows_removed} rows with too many null values")

# Filter for Released Movies Only 
print("\n Filtering for 'Released' movies only...")
print(f"   Status distribution before filter:")
print(df['status'].value_counts())
df = df[df['status'] == 'Released']
print(f"Kept only 'Released' movies. Remaining: {len(df)} movies")

# Drop the status column as it's no longer needed
#df = df.drop(columns=['status'])
#print(f"Dropped 'status' column")


print("Data cleaning and transformation complete!")
print(f"\nFinal DataFrame shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")


   Converting 'budget', 'id', 'popularity' to numeric...
   Converting 'release_date' to datetime...
Data types converted successfully!

 Replacing unrealistic/placeholder values...
Replacing 0 values with NaN for budget, revenue, runtime...
   - Budget: 0 null values
   - Revenue: 0 null values
   - Runtime: 0 null values

 Converting budget and revenue to millions of USD...
   Created 'budget_musd' and 'revenue_musd' columns
   Sample budget (M USD): [356.0, 237.0, 245.0]
   Sample revenue (M USD): [2799.4391, 2923.706026, 2068.223624]

 Replacing placeholder text in 'overview' and 'tagline'...
   - Overview null values: 0
   - Tagline null values: 0

 Removing duplicates...
Removed 0 duplicate rows based on id

 Dropping rows with missing 'id' or 'title'...
Removed 0 rows with missing id/title

 Keeping only rows with at least 10 non-null columns...
Removed 0 rows with too many null values

 Filtering for 'Released' movies only...
   Status distribution before filter:
status
Release

In [83]:
print(df)

                       backdrop_path                belongs_to_collection  \
0   /7RyHsO4yDXtBv1zUU3mTpHeQ0d5.jpg              The Avengers Collection   
1   /vL5LR6WdxWPjLPFRLe133jXWsh5.jpg                    Avatar Collection   
2   /8BTsTfln4jlQrLXUBquXJ0ASQy9.jpg                 Star Wars Collection   
3   /mDfJG3LC3Dqb67AZ52x3Z0jU0uB.jpg              The Avengers Collection   
4   /tupgjqhWx5oieQrdyesO3aclUX9.jpg                                 None   
5   /dF6FjTZzRTENfB4R17HDN20jLT2.jpg             Jurassic Park Collection   
6   /4G7SzRAaXYZ5hYfS05wbTzjv2Tn.jpg    The Lion King (Reboot) Collection   
7   /9BBTo63ANSmhC4e6r62OJFuK2GL.jpg              The Avengers Collection   
8   /cHkhb5A4gQRK6zs6Pv7zorHs8Nk.jpg  The Fast and the Furious Collection   
9   /kIBK5SKwgqIIuRKhhWrJn3XkbPq.jpg              The Avengers Collection   
10  /b6ZJZHUdMEFECvGiDpJjlfUWela.jpg             Black Panther Collection   
11  /n5A7brJCjejceZmHyujwUTVgQNC.jpg              Harry Potter Collection   

Data Cleaning (Reodering Columns)

In [85]:
Reordered_columns = [
    'id', 'title', 'tagline', 'release_date', 'genres', 'belongs_to_collection',
    'original_language', 'budget_musd', 'revenue_musd', 'production_companies',
    'production_countries', 'vote_count', 'vote_average', 'popularity', 'runtime',
    'overview', 'spoken_languages', 'poster_path'
]

# Check which columns exist in our DataFrame
existing_columns = [col for col in Reordered_columns if col in df.columns]
missing_columns = [col for col in Reordered_columns if col not in df.columns]

print("\nChecking columns...")
print(f"Columns that exist: {len(existing_columns)}/{len(Reordered_columns)}")
if missing_columns:
    print(f"Missing columns (will be skipped): {missing_columns}")

# Also keep any extra columns that aren't in the desired list
extra_columns = [col for col in df.columns if col not in Reordered_columns and col not in ['budget', 'revenue']]
if extra_columns:
    print(f"Extra columns found: {extra_columns}")
    # Add them to the end
    final_columns = existing_columns + extra_columns
else:
    final_columns = existing_columns


df = df[final_columns]
print(f"Columns reordered successfully!")


df = df.reset_index(drop=True)
print(f"Index reset successfully!")


print(f"\nFinal DataFrame Summary:")
print(f"   - Shape: {df.shape}")
print(f"   - Columns: {len(df.columns)}")
print(f"   - Rows: {len(df)}")

print("\nColumn List:")
for i, col in enumerate(df.columns, 1):
    non_null = df[col].notna().sum()
    print(f"   {i:2d}. {col:25s} - {non_null}/{len(df)} non-null")

df.to_csv("tmdb_cleaned_data.csv", index=False)
print("Saved to 'tmdb_cleaned_data.csv'")



Checking columns...
Columns that exist: 18/18
Extra columns found: ['backdrop_path', 'origin_country', 'status']
Columns reordered successfully!
Index reset successfully!

Final DataFrame Summary:
   - Shape: (18, 21)
   - Columns: 21
   - Rows: 18

Column List:
    1. id                        - 18/18 non-null
    2. title                     - 18/18 non-null
    3. tagline                   - 18/18 non-null
    4. release_date              - 18/18 non-null
    5. genres                    - 18/18 non-null
    6. belongs_to_collection     - 16/18 non-null
    7. original_language         - 18/18 non-null
    8. budget_musd               - 18/18 non-null
    9. revenue_musd              - 18/18 non-null
   10. production_companies      - 18/18 non-null
   11. production_countries      - 18/18 non-null
   12. vote_count                - 18/18 non-null
   13. vote_average              - 18/18 non-null
   14. popularity                - 18/18 non-null
   15. runtime                   - 1

In [86]:
display(df.head())

,id,title,tagline,release_date,genres,belongs_to_collection,original_language,budget_musd,revenue_musd,production_companies,...,vote_count,vote_average,popularity,runtime,overview,spoken_languages,poster_path,backdrop_path,origin_country,status
0,299534,Avengers: Endgame,Avenge the fallen.,2019-04-24,Adventure|Science Fiction|Action,The Avengers Collection,en,356.0,2799.439100,Marvel Studios,...,26842,8.238,16.8333,181.0,After the devastating events of Avengers: Infi...,English|Japanese|Xhosa,/ulzhLuWrPK07P1YkdWQLZnQh1JL.jpg,/7RyHsO4yDXtBv1zUU3mTpHeQ0d5.jpg,[US],Released
1,19995,Avatar,Enter the world of Pandora.,2009-12-15,Action|Adventure|Fantasy|Science Fiction,Avatar Collection,en,237.0,2923.706026,Dune Entertainment|Lightstorm Entertainment|20...,...,32714,7.593,22.9456,162.0,"In the 22nd century, a paraplegic Marine is di...",English|Spanish,/gKY6q7SjCkAU6FqvqWybDYgUKIF.jpg,/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg,[US],Released
2,140607,Star Wars: The Force Awakens,Every generation has a story.,2015-12-15,Adventure|Action|Science Fiction,Star Wars Collection,en,245.0,2068.223624,Lucasfilm Ltd.|Bad Robot,...,20021,7.255,11.2076,136.0,Thirty years after defeating the Galactic Empi...,English,/wqnLdwVXoBjKibFRR5U3y0aDUhs.jpg,/8BTsTfln4jlQrLXUBquXJ0ASQy9.jpg,[US],Released
3,299536,Avengers: Infinity War,Destiny arrives all the same.,2018-04-25,Adventure|Action|Science Fiction,The Avengers Collection,en,300.0,2052.415039,Marvel Studios,...,31060,8.236,33.9464,149.0,As the Avengers and their allies have continue...,English|Xhosa,/7WsyChQLEftFiDOVTGkv3hFpyyt.jpg,/mDfJG3LC3Dqb67AZ52x3Z0jU0uB.jpg,[US],Released
4,597,Titanic,Nothing on earth could come between them.,1997-11-18,Drama|Romance,None,en,200.0,2264.162353,Paramount Pictures|20th Century Fox|Lightstorm...,...,26406,7.905,25.7291,194.0,101-year-old Rose DeWitt Bukater tells the sto...,English|French|German|Swedish|Italian|Russian,/9xjZS2rlVxm8SFx8kPC3aIGCOYQ.jpg,/tupgjqhWx5oieQrdyesO3aclUX9.jpg,[US],Released
